<a href="https://colab.research.google.com/github/centeno/covid-impact-scrapper/blob/master/step01_2_merge_grey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
# Manipulação e visualização de dados
import os
import re
import sys
import glob
import socket
import pandas as pd


#dataviz
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output 


# Definimos o parâmetro abaixo para evitar o erro:
# RecursionError: maximum recursion depth exceeded
# Ao gravar os resultados em disco
sys.setrecursionlimit(50000)

%matplotlib inline

In [ ]:
#import os
#os.chdir("/content/drive/My Drive/covid19impact/data/2020/01_collect/en/")
os.chdir("/home/eluis/workspace/covid19impact/data/01_grey/2020/01_collect/en/")

#!ls

# Input

## Merge csv

In [ ]:
#path = "/content/drive/My Drive/covid19impact/data/2021_all/01_collect/en/"
path = "/home/eluis/workspace/covid19impact/data/01_grey/2020/01_collect/en/"

In [ ]:
#os.chdir("/mydir")
os.chdir(path)

In [ ]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [ ]:
len(all_filenames)

458

In [ ]:
#combine all files in the list
df = pd.concat([pd.read_csv(f, sep=';', parse_dates=["date"], engine="python", encoding='UTF-8') for f in all_filenames ])

In [ ]:
print(df.shape);
df.head()

(20068, 25)


,Unnamed: 0,date,header,summary,link,protocol,domain,path,query,fragment,...,CountryName,State,City,Longitude,Latitude,IpAddress,header_context_true,summary_context_true,Asn,edges
0,0,2020-01-22,Six disaster recovery pitfalls and how to avoi...,"Jan 23, 2020 — We look at some key pitfalls in...",https://www.computerweekly.com/feature/Six-dis...,https,www.computerweekly.com,/feature/Six-disaster-recovery-pitfalls-and-ho...,NaN,NaN,...,United States,Louisiana,Reddell,-92.426422,30.675011,206.19.49.153,Six disaster recovery pitfalls and how to avoi...,"Jan 23, 2020 — We look at some key pitfalls in...",NaN,NaN
1,1,2020-01-22,Customer Service Executive at Swift Networks L...,"Jan 23, 2020 — ... Training/ Mentorship · Soft...",https://job247sure.com/jobs/customer-service-e...,https,job247sure.com,/jobs/customer-service-executive-at-swift-netw...,NaN,NaN,...,Germany,Saarland,Saarbrücken,7.000000,49.233299,51.89.21.15,Customer Service Executive at Swift Networks L...,"Jan 23, 2020 — ... Training/ Mentorship · Soft...",NaN,NaN
2,2,2020-01-22,Grant Thornton Zim successfully revives two tr...,"Jan 22, 2020 — Local IT firmssay they tend to ...",https://newziana.co.zw/2020/01/22/grant-thornt...,https,newziana.co.zw,/2020/01/22/grant-thornton-zim-successfully-re...,NaN,NaN,...,South Africa,Gauteng,Johannesburg,28.056391,-26.199169,129.232.213.100,Grant Thornton Zim successfully revives two tr...,"Jan 22, 2020 — Local IT firmssay they tend to ...",NaN,NaN
3,3,2020-01-22,Terms of Service - Wilson's Media,"... agent, agile software development, aging, ...",https://www.wilsonsmedia.com/terms-of-service/,https,www.wilsonsmedia.com,/terms-of-service/,NaN,NaN,...,Ireland,NaN,NaN,-8.000000,53.000000,144.217.146.128,Terms of Service - Wilson's Media,"... agent, agile software development, aging, ...",NaN,NaN
4,4,2020-01-22,Akwajobs · Program Manager ERD at Internationa...,"Jan 22, 2020 — ... is to help the people whose...",http://www.akwajobs.com/job/program-manager-er...,http,www.akwajobs.com,/job/program-manager-erd-at-international-resc...,NaN,NaN,...,United States,Pennsylvania,Philadelphia,-75.144279,39.948841,69.164.198.51,Akwajobs · Program Manager ERD at Internationa...,"Jan 22, 2020 — ... is to help the people whose...",NaN,NaN


In [ ]:
# Delete columns
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
#df = df.drop_duplicates(subset=['header', 'summary'], keep='first')
df['status2-duplicated'] = df.duplicated(subset=['header','summary','link'])

In [ ]:
print(df.shape);
df[['header', 'status2-duplicated']].head()

(20068, 25)


,header,status2-duplicated
0,Six disaster recovery pitfalls and how to avoi...,False
1,Customer Service Executive at Swift Networks L...,False
2,Grant Thornton Zim successfully revives two tr...,False
3,Terms of Service - Wilson's Media,False
4,Akwajobs · Program Manager ERD at Internationa...,False


In [ ]:
df[df['status2-duplicated']==True].shape

(8957, 25)

# GEOINT

In [ ]:
import plotly.express as px 
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", zoom = 0)
fig.update_layout(mapbox_style="open-street-map") 
#fig.show()

# build jupyter dash app 
app = JupyterDash(__name__)
# add html components and figure to app
app.layout = html.Div([dcc.Graph(figure=fig)])
# run app inline
app.run_server(mode='inline')

# Output

In [ ]:
path2 = "/home/eluis/workspace/covid19impact/data/01_grey/2020/02_analytics/"

In [ ]:
df.to_csv(path2 +  'step01-2_merge_grey_v1.csv', sep=';', encoding='UTF-8')